In [7]:
using Plots
using PyCall
using ControlSystems

gr()

Plots.GRBackend()

In [8]:
zc=0.8
g=9.8
dt=0.001

#支持時間
Tsup=Int32(0.8*1000)
#全脚期間
Tdl=0.2

#全体のステップ数
k_max=Int32(1+(Tsup*6))
#目標予見ステップ
M=1000

#支持脚位置
sx=[0.0 0.0 0.3 0.6 0.9 0.9]
sy=[0.0 0.2 0.0 0.2 0.0 0.2]


#目標ZMPベクトル
px_ref=[[sx[1] for s in 1:Tsup]
        [sx[2] for s in 1:Tsup]
        [sx[3] for s in 1:Tsup]
        [sx[4] for s in 1:Tsup]
        [sx[5] for s in 1:Tsup]
        [sx[6] for s in 1:Tsup+1+M]]

py_ref=[[sy[1] for s in 1:Tsup]
        [sy[2] for s in 1:Tsup]
        [sy[3] for s in 1:Tsup]
        [sy[4] for s in 1:Tsup]
        [sy[5] for s in 1:Tsup]
        [sy[6] for s in 1:Tsup+1+M]]

#計算上のZMPベクトル
px=zeros(Float64,k_max) 
py=zeros(Float64,k_max) 

4801-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮  
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [9]:
A=[1 dt dt^2/2
    0 1 dt
    0 0 1]
B=[dt^3/6
    dt^2/2
    dt]
C=[1 0 -zc/g]

Q=[100 0 0 0
    0 10 0 0
    0 0 1 0
    0 0 0 1]
R=1.0

#エラーシステム

#内部状態ベクトル
x=[0.0
    0.0
    0.0]

y=[0.01
    0.0
    0.0]

φ=[1 -C*A
    zeros(Float64,3) A]
G=[-C*B
    B]
Gr=[1.0
    0.0
    0.0
    0.0]

4-element Array{Float64,1}:
 1.0
 0.0
 0.0
 0.0

In [10]:
k=0

t=[i for i in 1:k_max]
x_plot_log=zeros(Float64,k_max)
x_plot_log[k+1]=x[1] #plotするために記録
xv_plot_log=zeros(Float64,k_max)
xv_plot_log[k+1]=x[2]
xa_plot_log=zeros(Float64,k_max)
xa_plot_log[k+1]=x[3]

t=[i for i in 1:k_max]
y_plot_log=zeros(Float64,k_max)
y_plot_log[k+1]=y[1] #plotするために記録
yv_plot_log=zeros(Float64,k_max)
yv_plot_log[k+1]=y[2]
ya_plot_log=zeros(Float64,k_max)
ya_plot_log[k+1]=y[3] 


xdu=0
xdR=px_ref[k+1]-px[1] #dR[K+1]
ydu=0
ydR=py_ref[k+1]-py[1] #dR[K+1]

xe=px[1]-C*x
ye=py[1]-C*y


dx=[0; 0; 0]
dy=[0; 0; 0]

X=φ*[xe; dx]+G*xdu+Gr*xdR
Y=φ*[ye; dy]+G*ydu+Gr*ydR


old_x=x
old_y=y

xu=0
yu=0

P=dare(φ,G,Q,R)

ξ=(eye(4)-G*(R+G'*P*G)[1]*G'*P)*φ

xsum=0
ysum=0
for j in 1:M
    Fr=-(R+G'*P*G)*G'*(ξ')^(j-1)*P*Gr
    xsum +=Fr*(px_ref[k+j]-px_ref[k+j-1+1]) #K=0のみ+1
    ysum +=Fr*(py_ref[k+j]-py_ref[k+j-1+1]) #K=0のみ+1
end
F=-(R+G'*P*G)*G'*P*φ
xdu=F*X+xsum
ydu=F*Y+ysum

xu+=xdu
yu+=ydu

1-element Array{Float64,1}:
 -0.107308

In [11]:
for i in 1:k_max-1  
    #求めた入力を用いてk+1の状態を計算する
    px[k+1]=(C*x)[1]
    py[k+1]=(C*y)[1]
    
    x=A*x+B*xu[1]
    y=A*y+B*yu[1]
    
    k+=1
    
    x_plot_log[k+1]=x[1] #plotするために記録
    xv_plot_log[k+1]=x[2]
    xa_plot_log[k+1]=x[3]
    
    y_plot_log[k+1]=y[1] #plotするために記録
    yv_plot_log[k+1]=y[2]
    ya_plot_log[k+1]=y[3]
    
    dx=x-old_x
    dy=y-old_y
    
    xdR=px_ref[k+1]-px_ref[k]
    ydR=py_ref[k+1]-py_ref[k]
    
    
    xe=px_ref[k]-C*x
    ye=py_ref[k]-C*y
    
    X=φ*[xe; dx]+G*xdu[1]+Gr*xdR
    Y=φ*[ye; dy]+G*ydu[1]+Gr*ydR
    
    old_x=x
    old_y=y
    
    xsum=0
    ysum=0
    for j in 1:M
        Fr=-(R+G'*P*G)*G'*(ξ')^(j-1)*P*Gr
        xsum +=Fr*(px_ref[k+j]-px_ref[k+j-1])
        ysum +=Fr*(py_ref[k+j]-py_ref[k+j-1])
    end
    F=-(R+G'*P*G)*G'*P*φ
    xdu=F*X+xsum
    ydu=F*Y+ysum
    
    xu+=xdu
    yu+=ydu
end
    